In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
import pymysql

# EXTRACT DATA

In [2]:
#Read data from science journal
#assets/data/
scienceJournal_plasticWasteData = pd.read_csv("2014JambeckStudyDataTransformed.csv")
country_geocode=pd.read_csv("country_geocodes.csv")
country_abbreviations=pd.read_csv("country_abbreviations.csv")

# TRANSFORM DATA

In [3]:

#This is the information that was released from 2014 and it's for the year 2010
scienceJournal_data_2014=pd.DataFrame(scienceJournal_plasticWasteData)
scienceJournal_data_2014.head()

,country,economic_status,coastal_population,waste_generation_rate_kg_person_day,percentage_plastic_in_waste_stream,percentage_inadequately_managed_waste,percent_littered_waste,waste_generation_kg_day,plastic_waste_generation_kg_day,inadequately_managed_plastic waste_kg_day,plastic_waste_littered_kg_day,mismanaged_plastic_waste_kg_person_day,mismanaged_plastic_waste_in_2010_tonnes,mismanaged_plastic_waste_in_2025_tonnes
0,Albania,LMI,2530533,0.77,9,45,2.0,1948510,174392,77897,3488,0.0322,29705,63051
1,Algeria,UMI,16556580,1.20,12,58,2.0,19867896,2374214,1378693,47484,0.0861,520555,1017444
2,Angola,LMI,3790041,0.48,13,71,2.0,1819220,235589,166597,4712,0.0452,62528,136770
3,Anguilla,HIC,14561,2.10,12,2,2.0,30578,3654,68,73,0.0097,52,73
4,Antigua and Barbuda,HIC,66843,5.50,12,6,2.0,367637,43933,2555,879,0.0514,1253,1385


In [5]:
#Table One - Static as it's specific to the science jouranl results from 2014
plasticwaste_study_2014=scienceJournal_data_2014.drop(columns=["mismanaged_plastic_waste_in_2010_tonnes", "mismanaged_plastic_waste_in_2025_tonnes"])
plasticwaste_study_2014.head()

,country,economic_status,coastal_population,waste_generation_rate_kg_person_day,percentage_plastic_in_waste_stream,percentage_inadequately_managed_waste,percent_littered_waste,waste_generation_kg_day,plastic_waste_generation_kg_day,inadequately_managed_plastic waste_kg_day,plastic_waste_littered_kg_day,mismanaged_plastic_waste_kg_person_day
0,Albania,LMI,2530533,0.77,9,45,2.0,1948510,174392,77897,3488,0.0322
1,Algeria,UMI,16556580,1.20,12,58,2.0,19867896,2374214,1378693,47484,0.0861
2,Angola,LMI,3790041,0.48,13,71,2.0,1819220,235589,166597,4712,0.0452
3,Anguilla,HIC,14561,2.10,12,2,2.0,30578,3654,68,73,0.0097
4,Antigua and Barbuda,HIC,66843,5.50,12,6,2.0,367637,43933,2555,879,0.0514


In [6]:
#Table Two - mismanaged_plastic_waste by year, we only have 2010
mismanaged_plastic_wasteYearly=scienceJournal_data_2014[["country","mismanaged_plastic_waste_in_2010_tonnes"]]

In [7]:
#Table Three - mismanaged_plastic_waste by year projections 
mismanaged_plastic_waste_projections=scienceJournal_data_2014[["country","mismanaged_plastic_waste_in_2025_tonnes"]]

In [8]:
#Table Four - 2014 Study Analytics and Calculations

#Calculate changed from 2010 to 2025
percent_change_2010_to_2025=round((scienceJournal_data_2014["mismanaged_plastic_waste_in_2025_tonnes"]-scienceJournal_data_2014["mismanaged_plastic_waste_in_2010_tonnes"])/scienceJournal_data_2014["mismanaged_plastic_waste_in_2010_tonnes"],2)
percent_change_2010_to_2025=pd.DataFrame(percent_change_2010_to_2025)

#Subtracting managed plastic = plastic generated -ineaquately-litter
managed_plastic_waste=scienceJournal_data_2014["plastic_waste_generation_kg_day"]-scienceJournal_data_2014["inadequately_managed_plastic waste_kg_day"]-scienceJournal_data_2014["plastic_waste_littered_kg_day"]
managed_plastic_waste=pd.DataFrame(managed_plastic_waste)

#Adding mismanaged = ineaquately+litter
mismanaged_plastic_waste=scienceJournal_data_2014["inadequately_managed_plastic waste_kg_day"]+scienceJournal_data_2014["plastic_waste_littered_kg_day"]
mismanaged_plastic_waste=pd.DataFrame(mismanaged_plastic_waste)

analytics_df=scienceJournal_data_2014[["country"]]
analytics_df["mismanagedPlastic_change_2010_2025"]=percent_change_2010_to_2025
analytics_df["managed_plasticWaste_kg_day"]=managed_plastic_waste
analytics_df["mismanaged_plasticWaste_kg_day"]=mismanaged_plastic_waste

C:\Users\aruba\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\aruba\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\aruba\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [9]:
#Find the percentage amount 
managed_plasticWaste_kg_day_percentage= round(analytics_df["managed_plasticWaste_kg_day"]/scienceJournal_data_2014["plastic_waste_generation_kg_day"]*100,0)
managed_plasticWaste_kg_day_percentage=pd.DataFrame(managed_plasticWaste_kg_day_percentage)

mismanaged_plasticWaste_kg_day_percentage=round(analytics_df["mismanaged_plasticWaste_kg_day"]/scienceJournal_data_2014["plastic_waste_generation_kg_day"]*100,0)
mismanaged_plasticWaste_kg_day_percentage=pd.DataFrame(mismanaged_plasticWaste_kg_day_percentage)


In [10]:
#Find the per person impact
#1 kg = 2.20462
#plastic to ocean = .25
kgconvert=2.20462
factor_platic_to_ocean=.25

#Convert this number to pounds
mismanaged_plastic_pperson_lb=round(scienceJournal_data_2014["mismanaged_plastic_waste_kg_person_day"]*kgconvert,2)
mismanaged_plastic_pperson_lb=pd.DataFrame(mismanaged_plastic_pperson_lb)

#Calculate how much goes directly into the ocean 
mismanaged_plastic_pperson_lb_to_ocean=round(mismanaged_plastic_pperson_lb*factor_platic_to_ocean,2)
mismanaged_plastic_pperson_lb_to_ocean=pd.DataFrame(mismanaged_plastic_pperson_lb_to_ocean)

In [11]:
platicwaste_2014study_calculations=analytics_df
platicwaste_2014study_calculations=pd.DataFrame(platicwaste_2014study_calculations)

#Add the rest of the calculations 
platicwaste_2014study_calculations["managed_plasticWaste_kg_day_percentage"]=managed_plasticWaste_kg_day_percentage
platicwaste_2014study_calculations["mismanaged_plasticWaste_kg_day_percentage"]=mismanaged_plasticWaste_kg_day_percentage
platicwaste_2014study_calculations["mismanaged_plastic_pperson_lb_to_ocean_day"]=mismanaged_plastic_pperson_lb_to_ocean

platicwaste_2014study_calculations.head()


,country,mismanagedPlastic_change_2010_2025,managed_plasticWaste_kg_day,mismanaged_plasticWaste_kg_day,managed_plasticWaste_kg_day_percentage,mismanaged_plasticWaste_kg_day_percentage,mismanaged_plastic_pperson_lb_to_ocean_day
0,Albania,1.12,93007,81385,53.0,47.0,0.02
1,Algeria,0.95,948037,1426177,40.0,60.0,0.05
2,Angola,1.19,64280,171309,27.0,73.0,0.02
3,Anguilla,0.40,3513,141,96.0,4.0,0.00
4,Antigua and Barbuda,0.11,40499,3434,92.0,8.0,0.03


# LOAD

In [12]:
engine = create_engine("sqlite:///DRAFT.sqlite")
conn = engine.connect()

In [13]:
#Tables

#Static
plasticwaste_study_2014.to_sql(name="plasticwaste_study_2014", con= engine, if_exists="append", index=False)
country_geocode.to_sql(name="country_geocode", con= engine, if_exists="append", index=False)
country_abbreviations.to_sql(name="country_abbreviations", con= engine, if_exists="append", index=False)

#Open
mismanaged_plastic_wasteYearly.to_sql(name="mismanaged_plastic_wasteYearly", con= engine, if_exists="append", index=False)
mismanaged_plastic_waste_projections.to_sql(name="mismanaged_plastic_waste_projections", con= engine, if_exists="append", index=False)
platicwaste_2014study_calculations.to_sql(name="platicwaste_2014study_calculations", con= engine, if_exists="append", index=False)